In [1]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [2]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Load config from YAML file
config = AnalyzeConfig.from_yaml(
    "/Users/ryanarman/code/oumi/configs/examples/analyze/analyze_banking77.yaml"
)

# Override settings for this run
dataset_path = "/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl"
config.dataset_path = dataset_path
config.dataset_name = None  # Clear dataset_name so it uses dataset_path instead
config.sample_count = 10
config.chat_template = "chat_ml"

# Set absolute output path (makes it easier to find the results!)
config.output_path = "/Users/ryanarman/code/oumi/analysis_output/banking77_final"

print(f"✅ Config loaded with {len(config.analyzers)} analyzers:")
for analyzer in config.analyzers:
    instance_id = analyzer.instance_id or analyzer.id
    print(f"  - {instance_id} (type: {analyzer.id})")

print(f"📁 Output will be saved to: {config.output_path}")

# Validate the configuration
config.finalize_and_validate()
print("✅ Config validated successfully!")

analyzer = DatasetAnalyzer(config)

✅ Config loaded with 1 analyzers:
  - length (type: length)
📁 Output will be saved to: /Users/ryanarman/code/oumi/analysis_output/banking77_final
✅ Config validated successfully!
[2026-01-06 11:51:52,151][oumi][rank0][pid:50829][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2026-01-06 11:51:52,152][oumi.utils.analysis_utils][rank0][pid:50829][MainThread][INFO]][analysis_utils.py:227] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2026-01-06 11:51:52,153][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2026-01-06 11:51:52,283][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:318] Initialized sample analyzer: length


In [3]:
# Run the analysis
analyzer.analyze_dataset()

# The results are stored in analyzer object
if analyzer._analysis_results:
    print(
        f"Total conversations analyzed: {analyzer._analysis_results.conversations_analyzed}"
    )

[2026-01-06 11:51:52,291][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:361] Starting analysis of dataset: None
[2026-01-06 11:51:52,292][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:362] Using 1 sample analyzers: ['length']
[2026-01-06 11:51:52,293][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:385] Analyzing 10 of 8002 conversations
[2026-01-06 11:51:52,293][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:435] Converting conversation dataset with 8002 items
[2026-01-06 11:51:52,293][oumi][rank0][pid:50829][MainThread][INFO]][dataset_analyzer.py:442] Limiting analysis to first 10 items (dataset has 8002 total)


Converting Unknown Dataset to DataFrames: 100%|██████████| 10/10 [00:00<00:00, 511.56item/s]


[2026-01-06 11:51:52,334][oumi.utils.analysis_utils][rank0][pid:50829][MainThread][INFO]][analysis_utils.py:1322] Adding default schema entries for 2 columns not in base schema: ['label', 'label_name']
Total conversations analyzed: 10


In [4]:
analyzer.analysis_df

,conversation_index,conversation_id,num_messages,conversation_text_content,label,label_name,conversation_text_content__length__token_count,message_index,message_id,role,text_content,text_content__length__token_count
0,0,0,3,<|im_start|>system\nYou are a banking intent c...,52,request_refund,1806,0,msg_0,system,You are a banking intent classifier. Classify ...,1751
1,0,0,3,<|im_start|>system\nYou are a banking intent c...,52,request_refund,1806,1,msg_1,user,"If I bought something I didn't like, can I get...",14
2,0,0,3,<|im_start|>system\nYou are a banking intent c...,52,request_refund,1806,2,msg_2,assistant,52,1
3,1,1,3,<|im_start|>system\nYou are a banking intent c...,69,verify_my_identity,1805,0,msg_0,system,You are a banking intent classifier. Classify ...,1751
4,1,1,3,<|im_start|>system\nYou are a banking intent c...,69,verify_my_identity,1805,1,msg_1,user,"If I'm getting my identity verified, what all ...",13
5,1,1,3,<|im_start|>system\nYou are a banking intent c...,69,verify_my_identity,1805,2,msg_2,assistant,69,1
6,2,2,3,<|im_start|>system\nYou are a banking intent c...,59,top_up_failed,1801,0,msg_0,system,You are a banking intent classifier. Classify ...,1751
7,2,2,3,<|im_start|>system\nYou are a banking intent c...,59,top_up_failed,1801,1,msg_1,user,My top-up was rejected by an app.,9
8,2,2,3,<|im_start|>system\nYou are a banking intent c...,59,top_up_failed,1801,2,msg_2,assistant,59,1
9,3,3,3,<|im_start|>system\nYou are a banking intent c...,54,supported_cards_and_currencies,1807,0,msg_0,system,You are a banking intent classifier. Classify ...,1751


In [ ]:
from oumi.core.analyze.report_generator import HTMLReportGenerator

# Save the analysis results to disk
import json
import traceback
from pathlib import Path

# Create output directory
output_dir = Path(config.output_path)
output_dir.mkdir(parents=True, exist_ok=True)

report_gen = HTMLReportGenerator()
report_path = report_gen.generate_report(
    analyzer,
    output_path=output_dir,
    title=config.report_title or "Banking77 Analysis Report",
)

UndefinedError: 'int object' has no attribute 'mean'

In [ ]:
# Save message-level results
if analyzer.message_df is not None and not analyzer.message_df.empty:
    msg_path = output_dir / "messages_df.parquet"
    analyzer.message_df.to_parquet(msg_path)
    print(f"✅ Saved message analysis to: {msg_path}")

# Save conversation-level results
if analyzer.conversation_df is not None and not analyzer.conversation_df.empty:
    conv_path = output_dir / "conversations_df.parquet"
    analyzer.conversation_df.to_parquet(conv_path)
    print(f"✅ Saved conversation analysis to: {conv_path}")

# Save merged results
if analyzer.analysis_df is not None and not analyzer.analysis_df.empty:
    merged_path = output_dir / "merged_df.parquet"
    analyzer.analysis_df.to_parquet(merged_path)
    print(f"✅ Saved merged analysis to: {merged_path}")

# Save analysis summary
summary_path = output_dir / "analysis_summary.json"
with open(summary_path, "w") as f:
    json.dump(analyzer.analysis_summary, f, indent=2, default=str)
print(f"✅ Saved analysis summary to: {summary_path}")

# Generate HTML report if configured
if config.generate_report:
    try:
        from oumi.core.analyze.report_generator import HTMLReportGenerator

        report_gen = HTMLReportGenerator()
        report_path = report_gen.generate_report(
            analyzer,
            output_path=output_dir,
            title=config.report_title or "Banking77 Analysis Report",
        )
        print(f"✅ Generated HTML report at: {report_path / 'index.html'}")
    except ImportError:
        print("⚠️  Plotly not installed. Skipping HTML report generation.")
        print("   Install with: pip install 'oumi[analyze_advanced]'")
    except Exception as e:
        print(f"⚠️  Failed to generate HTML report: {e}")
        print("\n🔍 FULL TRACEBACK:")
        print("=" * 70)
        traceback.print_exc()
        print("=" * 70)

print(f"\n📁 All results saved to: {output_dir.absolute()}")


✅ Saved message analysis to: /Users/ryanarman/code/oumi/analysis_output/banking77_final/messages_df.parquet
✅ Saved conversation analysis to: /Users/ryanarman/code/oumi/analysis_output/banking77_final/conversations_df.parquet
✅ Saved merged analysis to: /Users/ryanarman/code/oumi/analysis_output/banking77_final/merged_df.parquet
✅ Saved analysis summary to: /Users/ryanarman/code/oumi/analysis_output/banking77_final/analysis_summary.json
⚠️  Failed to generate HTML report: type Undefined doesn't define __round__ method

🔍 FULL TRACEBACK:


Traceback (most recent call last):
  File "/var/folders/pz/hnk9b_tx2xz7fqsy0q7t4mj00000gn/T/ipykernel_50431/407050388.py", line 40, in <module>
    report_path = report_gen.generate_report(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ryanarman/code/oumi/src/oumi/core/analyze/report_generator.py", line 198, in generate_report
    html_content = self._template.render(**template_data)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/jinja2/environment.py", line 1295, in render
    self.environment.handle_exception()
  File "/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/jinja2/environment.py", line 942, in handle_exception
    raise rewrite_traceback_stack(source=source)
  File "/Users/ryanarman/code/oumi/src/oumi/core/analyze/templates/report_template.html.jinja", line 1054, in top-level template code
    <td>{{ stats.mean | round(2) }}</td>
    ^^^^^^^^^^^^^^^^^^^^^^^


📁 All results saved to: /Users/ryanarman/code/oumi/analysis_output/banking77_final


In [ ]:
summary = analyzer.analysis_summary
recommendations = summary["recommendations"]
recommendations


[{'category': 'warning',
  'severity': 'high',
  'title': 'Empty or near-empty messages detected',
  'description': 'Found 10 messages (33.3%) with 5 or fewer characters. These may indicate data quality issues or placeholder content that should be reviewed.',
  'affected_samples': 10,
  'metric_name': 'text_content',
  'threshold': 5.0,
  'details': {'empty_count': 10},
  'sample_indices': [2, 5, 8, 11, 14, 17, 20, 23, 26, 29]},
 {'category': 'warning',
  'severity': 'low',
  'title': 'Duplicate content detected',
  'description': 'Found 2 messages (10.0%) that are exact duplicates (excluding system prompts), representing 1 unique repeated texts. Consider deduplicating your dataset to improve training diversity.',
  'affected_samples': 2,
  'metric_name': 'text_content',
  'threshold': 0.05,
  'details': {'duplicate_count': 2,
   'unique_duplicated_texts': 1,
   'duplicate_percentage': 10.0},
  'sample_indices': [2, 20]}]

In [ ]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 12
len(merged_columns): 12


In [ ]:
analyzer_names = [a.instance_id for a in config.analyzers]
analyzer_names

['length']

# Conv level

In [ ]:
conv_df = analyzer.conversation_df
conv_df.head()

,conversation_index,conversation_id,num_messages,conversation_text_content,label,label_name,conversation_text_content__length__token_count
0,0,0,3,<|im_start|>system\nYou are a banking intent c...,52,request_refund,1806
1,1,1,3,<|im_start|>system\nYou are a banking intent c...,69,verify_my_identity,1805
2,2,2,3,<|im_start|>system\nYou are a banking intent c...,59,top_up_failed,1801
3,3,3,3,<|im_start|>system\nYou are a banking intent c...,54,supported_cards_and_currencies,1807
4,4,4,3,<|im_start|>system\nYou are a banking intent c...,57,top_up_by_card_charge,1804


In [ ]:
conv_df.conversation_text_content__helpfulness__score

AttributeError: 'DataFrame' object has no attribute 'conversation_text_content__helpfulness__score'

In [ ]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(conv_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])
    # print("\nInput:")
    # print(f"source_column: {info.source_column}")
    # print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: length
metric: token_count
description: Token count for conversation_text_content
value: 1806




# Message level

In [ ]:
msg_df = analyzer.message_df
msg_df.head()

,conversation_index,conversation_id,message_index,message_id,role,text_content,text_content__length__token_count
0,0,0,0,msg_0,system,You are a banking intent classifier. Classify ...,1751
1,0,0,1,msg_1,user,"If I bought something I didn't like, can I get...",14
2,0,0,2,msg_2,assistant,52,1
3,1,1,0,msg_0,system,You are a banking intent classifier. Classify ...,1751
4,1,1,1,msg_1,user,"If I'm getting my identity verified, what all ...",13


In [ ]:
msg_df[msg_df.role == "assistant"].text_content__response_quality__score

AttributeError: 'DataFrame' object has no attribute 'text_content__response_quality__score'

In [ ]:
msg_df[msg_df.role == "system"].text_content__instruction_quality__score.iloc[0]


AttributeError: 'DataFrame' object has no attribute 'text_content__instruction_quality__score'

In [ ]:
msg_df[msg_df.role == "system"].text_content__instruction_quality__label.iloc[0]


AttributeError: 'DataFrame' object has no attribute 'text_content__instruction_quality__label'

In [ ]:
msg_df[msg_df.role == "system"].text_content__instruction_quality__reasoning.iloc[0]


AttributeError: 'DataFrame' object has no attribute 'text_content__instruction_quality__reasoning'

In [ ]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

msg_columns = analyzer.message_df.columns
row = analyzer.message_df.iloc[0]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[3]


filtered_cols = filter_analyzer_columns(msg_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])

    print("\nInput:")
    print(f"[{row['role']}]: {row[info.source_column]}\n")
    # print(f"source_column: {info.source_column}")
    print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


IndexError: list index out of range